In [8]:
import cv2
from detecto import core, visualize

model = core.Model.load('kathakali.pth', ['Pathaaka','Mudraakhyam','Katakam','Mushti','Kartharee Mukham'])

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    # if not ret:
    #     break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    labels, boxes, scores = model.predict_top(frame)

    visualize.detect_live(model, score_filter=0.7)

cap.release()
cv2.destroyAllWindows()


[ WARN:0@271.000] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@271.001] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [18]:
import sys
import cv2
from PyQt5 import QtWidgets, QtCore, QtGui
from detecto import core, visualize

class CameraWidget(QtWidgets.QWidget):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.cap = cv2.VideoCapture(0)
        self.timer = QtCore.QTimer(self)
        self.timer.timeout.connect(self.nextFrameSlot)
        self.timer.start(1000./30)

    def nextFrameSlot(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            self.image = QtGui.QImage(frame, frame.shape[1], frame.shape[0], QtGui.QImage.Format_RGB888)
            self.update()

    def paintEvent(self, event):
        qp = QtGui.QPainter(self)
        qp.drawImage(0, 0, self.image)
        self.adjustSize()

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self, model, parent=None):
        super().__init__(parent)
        self.model = model
        self.centralwidget = QtWidgets.QWidget(self)
        self.setCentralWidget(self.centralwidget)
        self.cameraWidget = CameraWidget(self.centralwidget)
        self.cameraWidget.setMinimumSize(640, 480)
        self.cameraWidget.setMaximumSize(640, 480)
        self.cameraWidget.setObjectName("cameraWidget")
        self.horizontalLayout = QtWidgets.QHBoxLayout(self.centralwidget)
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.horizontalLayout.addWidget(self.cameraWidget)
        self.setFixedSize(self.size())
        self.exit_key = QtCore.Qt.Key_Q
        self.setFocusPolicy(QtCore.Qt.StrongFocus)

    def keyPressEvent(self, event):
        if event.key() == self.exit_key:
            self.close()

    def closeEvent(self, event):
        self.cameraWidget.cap.release()

if __name__ == "__main__":
    model = core.Model.load('kathakali.pth', ['Pathaaka', 'Mudraakhyam', 'Katakam', 'Mushti', 'Kartharee Mukham'])
    app = QtWidgets.QApplication(sys.argv)
    mainWin = MainWindow(model)
    mainWin.show()

    while True:
        ret, frame = mainWin.cameraWidget.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            labels, boxes, scores = model.predict_top(frame)
            visualize.show_labeled_image(frame, boxes, labels)
        QtWidgets.qApp.processEvents()


: 

: 